In [1]:
# import packages
import pandas as pd
import numpy as np
from google.cloud import bigquery
# # import plotly.graph_objects as go
# # import plotly.express as px
from datetime import datetime
# import json
import pprint

In [2]:
# Set plotly as plotting backend
# pd.options.plotting.backend = "plotly"


# Global variables

BQ_PROJECT = "bigquery-anwb"
BQ_DATASET_MASTER = "87223461"
BQ_DATASET_GA4_test = "analytics_290476086"
BQ_DATASET_GA4_prod = "analytics_229981733"

In [3]:
def lookup_bq_names(ga_name: str):
    ga_to_bq = {'user': 'fullVisitorId',
                'session': '''CONCAT(fullVisitorId, '-', cast(visitId AS string), '-', date)''',
                'date': 'date',
                'page': 'hits.page.pagePath',
                'event category': 'hits.eventInfo.eventCategory',
                'event action': 'hits.eventInfo.eventAction',
                'event label': 'hits.eventInfo.eventLabel',
                'campaign': 'trafficSource.campaign',
                'source': 'trafficSource.source',
                'medium': 'trafficSource.medium',
                'keyword': 'trafficSource.keyword',
                'device category': 'device.deviceCategory',
                'screen resolution': 'device.screenResolution',
                'browser': 'device.browser',
                'operating system': 'device.operatingSystem',
                'transactions': 'hits.transaction.transactionId',
                'affiliation': 'hits.transaction.affiliation',
                'revenue': ' hits.transaction.transactionRevenue / 1000000',
                'product': 'product.V2ProductName',
                'product revenue': 'product.productRevenue / 1000000',
                'product detail view': 'product.V2ProductName',
                }

    if ga_name.lower() in ga_to_bq:
        bq_name = ga_to_bq[ga_name.lower()]
    else:
        bq_name = ga_name

    return bq_name

In [4]:
def process_filters(filter_):
    if filter_['condition'] == 'regex':
        filter_['value'] = filter_['value'].replace('\\', '\\\\')
        filter_sql = f"""REGEXP_CONTAINS({filter_['dimension']},'{filter_['value']}')"""
    elif filter_['condition'] == 'in':
        filter_sql = f"""{filter_['dimension']} {filter_['condition']} ({filter_['value']})"""
    else:
        filter_sql = f"""{filter_['dimension']} {filter_['condition']} '{filter_['value']}'"""
    return filter_sql

In [5]:
def check_unnests(dimension):
    """"
    Bepaal welke items ge-unnest moeten worden
    """
    # @Robbert: dit moet mooier kunnen... toch??
    # Omdat ik nog niet weet hoe ik kan bepalen op welk niveau een customdimension is gedefinieerd,
    # wordt deze nog niet meegenomen in unnest

    unnest_list = []
    # if dimension.startswith('customDimensions'):
    #     unnest_list.append('unnest(customDimensions) as cd')
    if dimension.startswith('hits'):
        unnest_list.append('unnest(hits) as hits')
    # if dimension.startswith('hits.customDimensions'):
    #     unnest_list.append('unnest(hits) as hits')
    #     unnest_list.append('unnest(hits.customDimensions) as hits_cd')
    if dimension.startswith('product'):
        unnest_list.append('unnest(hits) as hits')
        unnest_list.append('unnest(hits.product) as product')
    if dimension.startswith('hits.promotion'):
        unnest_list.append('unnest(hits) as hits')
        unnest_list.append('unnest(hits.promotion) as promotion')
    return unnest_list

In [6]:
def set_funnel_scope(funnel_definitions):
    funnel_definitions['prep']['funnelScope'] = f"""{lookup_bq_names(funnel_definitions['funnelScope'])}"""

    return funnel_definitions


In [7]:
def set_funnel_dimensions(funnel_definitions):
    funnel_definitions['prep']['breakdownDimensions'] = [lookup_bq_names(x) for x in funnel_definitions['breakdownDimensions']]
    funnel_definitions['prep']['sql']['funnelDimensions'] = """
        ,  """.join(funnel_definitions['prep']['breakdownDimensions'])

    funnel_unnest = []
    for f in funnel_definitions['prep']['breakdownDimensions']:
        funnel_unnest += check_unnests(f)
    funnel_definitions['unnest'] = funnel_unnest

    return funnel_definitions

In [8]:
def set_funnel_date_range(funnel_definitions):
    date_from = funnel_definitions['dateRange'][0]
    date_to = funnel_definitions['dateRange'][1]
    funnel_definitions['prep']['dateFrom'] = datetime.strptime(date_from, '%m-%d-%Y').date().strftime("%Y%m%d")
    funnel_definitions['prep']['dateTo'] = datetime.strptime(date_to, '%m-%d-%Y').date().strftime("%Y%m%d")
    # print('set_funnel_date_range: ', funnel_definitions['sql'])

    return funnel_definitions

In [9]:
def set_funnel_filters(funnel_definitions):
    funnel_definitions['prep']['filters'] = funnel_definitions['filters']
    for f in funnel_definitions['prep']['filters']:

        f['dimension'] = lookup_bq_names(f['dimension'])

    funnel_filter_sql = [process_filters(f) for f in funnel_definitions['prep']['filters']]
    funnel_definitions['prep']['sql']['funnelFilters'] = """
    AND    """.join(funnel_filter_sql)

    return funnel_definitions

In [10]:
def set_funnel_steps(funnel_definitions):
    funnel_definitions['prep']['sql']['steps'] = []
    funnel_definitions['prep']['steps'] = funnel_definitions['steps']
    for step in funnel_definitions['prep']['steps']:
        sql_step_def = {}
        sql_step_def['stepId'] = step['stepId']
        s = []
        # voor iedere stap moeten zowel de benodigde unnest voor de hele funnel als de steps toegevoegd worden
        unnest = funnel_definitions['prep']['unnest']
        for f in step['step']:
            f['dimension'] = lookup_bq_names(f['dimension'])
            s.append(process_filters(f))
            step_sql = """
    AND    """.join(s)
            unnest += check_unnests(f['dimension'])

        sql_step_def['filters'] = step_sql
        unnest_dedup = list(dict.fromkeys(unnest))  # deduplicate list
        sql_step_def['unnest'] = """
                , """.join(unnest_dedup)
        funnel_definitions['prep']['sql']['steps'].append(sql_step_def)

    return funnel_definitions

In [11]:
def prep_sql(funnel_definitions):
    funnel_definitions['prep'] = {}
    funnel_definitions['prep']['sql'] = {}
    funnel_definitions['prep']['unnest'] = []
    funnel_definitions = set_funnel_scope(funnel_definitions)
    funnel_definitions = set_funnel_dimensions(funnel_definitions)
    funnel_definitions = set_funnel_date_range(funnel_definitions)
    funnel_definitions = set_funnel_filters(funnel_definitions)
    funnel_definitions = set_funnel_steps(funnel_definitions)

    for step in funnel_definitions['prep']['sql']['steps']:
        step['stepsql'] = f"""-- Step {step['stepId']}
    CREATE TEMP FUNCTION customDimensionByIndexUA(indx INT64, arr ARRAY<STRUCT<index INT64, value STRING>>) AS (
    (   SELECT x.value FROM UNNEST(arr) x WHERE indx=x.index)
    );
    SELECT distinct {funnel_definitions['prep']['funnelScope']} as id
        ,  visitStartTime + cast(hits.time/1000 as int64) as timestamp
        ,  EXTRACT(DATETIME FROM TIMESTAMP_SECONDS(visitStartTime + cast(hits.time/1000 as int64)) AT TIME ZONE "Europe/Amsterdam") as datetime
        ,  {funnel_definitions['prep']['sql']['funnelDimensions']}
    FROM   `{BQ_PROJECT}.{BQ_DATASET_MASTER}.ga_sessions_*` s, {step['unnest']}
    WHERE  _TABLE_SUFFIX >= '{funnel_definitions['prep']['dateFrom']}'
    AND    _TABLE_SUFFIX <= '{funnel_definitions['prep']['dateTo']}'
    AND    {funnel_definitions['prep']['sql']['funnelFilters']}
    AND    {step['filters']}"""

    return funnel_definitions['prep']

In [12]:
# step_dict = {...}
# df_count = []
# prev_step = None
# for step in prepped_definitions['steps']:
#     step_id = step['stepId']
#     df_step = get_UA_data(BQ_PROJECT, step_id, step['stepsql']) # haal data op
#     step_dict[step_id] = df_step.copy()
# #     print(step_id, prev_step_id)
# #     print(step_dict[step_id])
#     df_count = compare_steps('open', df_step.copy(), prev_step)
#     prev_step = df_step.copy()
#     print(step_dict[step_id]) # bij deze print zie ik de group kolom die in de functie heb toegevoegd. 
    
    

# def compare_steps( funnel_type, breakdown_dimensions, step, prev_step):
#     ...
#     if prev_step is None:
#         step['group'] = 'base'
# #         step_counts['base'] = df[df['match']]['id'].nunique()
#     else:
#         df = pd.merge(left=prev_step, right=step, how='outer',
#                       on=['id']+ pivot_list,
#                       suffixes=("_prev", ""),
#                       indicator=True)
#         df['group'] = np.where((df['_merge'] == 'left_only'), 'uitval', None)
#         ...
        
#     step_counts = step[pivot_list].value_counts()
#     return step_counts  

In [13]:
# SQL statement from session data
def get_UA_data(project: str, stepId, sql: str):
    # print(sql)
    bqclient = bigquery.Client(project=project)
    df = pd.read_csv(f'''{str(stepId)}.csv''', comment='#')
    # df = bqclient.query(sql).result().to_dataframe(create_bqstorage_client=True)
    df.to_csv(f'''{str(stepId)}.csv''', index=False)
    df.head()
    return df

In [14]:
def compare_steps(funnel_type, breakdown_dimensions, step_dict, step_id, prev_step_id):

    step_counts = {}
    step = step_dict[step_id].copy()

    pivot_list = [i.split('.', 1)[-1] for i in breakdown_dimensions]
    if prev_step_id is None:
        df = step
        df['step_id'] = step_id
        df['group'] = 'base'
    else:
        prev_step = step_dict[prev_step_id]
        df = pd.merge(left=prev_step, right=step, how='outer',
                      on=['id', 'breakdown'],
                      suffixes=("_prev", ""),
                      indicator=True)
        df['step_id'] = step_id
        df['group'] = np.where(
            (df['timestamp'] > df['timestamp_prev']) &
            (df['_merge'] == 'both'), 'base', None)

        if (funnel_type == 'open'):
            df['group'] = np.where(
                (df['_merge'] == 'right_only') & (df['group'].isna()), 'new', df['group'])
            df['group'] = np.where(
                (df['_merge'] == 'both') & (df['group'].isna()), 'new', df['group'])
    step_counts = pd.pivot_table(data=df, index=['step_id', 'breakdown', 'group'], values='id', aggfunc=pd.Series.nunique).reset_index()

    return step_counts

In [15]:
funnel_definitions = pd.read_json('funneldefinitions.json')
prepped_definitions = prep_sql(funnel_definitions['funnel'])
prev_step_id = None # geen best practice om df zo op te bouwen, maar werkt wel
step_dict = {}
funnel_count = pd.DataFrame()
prev_step = None
for step in prepped_definitions['sql']['steps']:
    step_id = step['stepId']
    df_step = get_UA_data(BQ_PROJECT, step_id, step['stepsql'])
    step_dict[step_id] = df_step

    df_count = compare_steps(funnel_definitions['funnel']['funnelType'],
                             prepped_definitions['breakdownDimensions'],
                             step_dict, step_id, prev_step_id)

    # funnel_count = pd.concat(funnel_count,df_count)
    funnel_count = funnel_count.append(df_count, ignore_index=True)

    prev_step_id = step_id

In [21]:
funnel_count.head()
#Grouping and perform count over each group
breakdown =  funnel_count['breakdown'].unique()
print(breakdown)

for i in breakdown:
    print(i)
    df = funnel_count[funnel_count['breakdown']==i].pivot(index='group', columns='step_id', values='id').reset_index()
    print(df)

['-']
-
step_id group      1     2     3
0        base  10328  6991  6892


In [40]:
df_test = df_step.head(10)
for c in df_test.columns:
    df_col = df_test[c]
    for r in df_col.items():
        print(r)

df_col.head()


KeyError: 'd'

In [17]:
breakdown =  funnel_count['breakdown'].unique()

funnel_pivot = funnel_count.pivot(index=['breakdown', 'group'], columns='step_id', values='id').reset_index()
breakdown = ['desktop']
for i in breakdown:
    print(i)
    df = funnel_pivot[funnel_pivot['breakdown']==i].drop(columns='breakdown')
    print(df)

desktop
Empty DataFrame
Columns: [group, 1, 2, 3]
Index: []


In [18]:
max_step = funnel_count["step_id"].max()
uitval = []
# uitval = df[df['group']=='base']
for i in range(1, max_step):
    x = df[df['group']=='base'][i].item() #- df[df['group']=='base'][i+1]
    uitval.append (df[df['group']=='base'][i].item() - df[df['group']=='base'][i+1].item())
    print(x)
print(uitval)


ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
for i in range(1, max_step):
    total = df[i].sum()
    uitval = df[df['group'] == 'base'][i].item() - df[df['group'] == 'base'][i + 1].item()
    perc_uitval = f'''{round((uitval/total)*100,1)}%''' 
    print(total, uitval, perc_uitval)

In [146]:
totals = [df[i].sum() for i in range(1,max_step + 1)]
print(totals)
uitval = [df[df['group']=='base'][i].item() - df[df['group']=='base'][i+1].item() for i in range(1,max_step)]
print(uitval)
perc = [(uitval[i]/ totals[i])*100 for i in range(max_step)]

[3.0, 3.0, 3.0]
[1.0, 0.0]


IndexError: list index out of range

In [134]:
[write_footer (i, 'total',df[i].sum()) for i in range(1,max_step + 1)]
[write_footer (i, 'uitval',df[i] - df[i+1]) for i in range(1,max_step)]


total - 1 - 3.0
total - 2 - 3.0
total - 3 - 3.0
uitval - 1 - 0    1.0
1    NaN
dtype: float64
uitval - 2 - 0    0.0
1    0.0
dtype: float64


[None, None]

In [117]:
def write_footer (step, title, value):
#     for i in values:
        print(f'''{title} - {step} - {value}''')